### TrumpTweet Notebook
This notebook references the code to create a Trump tweet. This includes:
- Processing the Trump Tweet archive to create a clean file of Trump's tweets
- Defining and training GRU long short term memory recurrent neural network
- Feeding data into the model in order to create a novel tweet

#### Data Prep

In [1]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [1]:
# Run the helpers scripts with the data and model helper objects
%run scripts/helpers
#from scripts import helpers

In [3]:
# Verify GPU support
tf.config.list_physical_devices('GPU')


[]

##### Create a datahelper and process raw data

In [2]:
# Create a datahelper object and designate the input file
dh = DataHelper(file_name='tweets_11-06-2020.csv')

# Prep the raw data to create the tweet file
dh.prep_raw_data(start_date='2020-08-01', end_date='2020-11-19')

# Print the number of tweets
print('The number of Tweets sent by Trump during the period is {}'.format(dh.num_tweets))


Data processing complete.
The number of Tweets sent by Trump during the period is 2362


##### Tokenize the text and create the dataset for model training

In [5]:
# Tokenize the text and create the dataset
dataset, tokenizer = dh.create_tokenizer('inputdata/clean_tweet.txt')
print('The number of unique characters is {0:,} and the dataset size is {1:,} document(s).' \
      ' The number of windows in the dataset for processing is {2:,}.'.format(dh.num_unique_chars,dh.dataset_size,dh.num_data_windows))


Dataset and tokenizer creation complete.
The number of unique characters is 75 and the dataset size is 1 document(s). The number of windows in the dataset for processing is 4,926,622.


##### Create the Model and Train It
The model is a stateless RNN made up of GRU cells

In [6]:
# Create the modelhelper object
mh = ModelHelper(epochs=20)

# Create the model
model = mh.create_model(tokenizer)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')

# Save a checkpoint after every epoch
EPOCHS = 20
checkpoint_filepath = 'checkpoints/weights.{epoch:02d}.hdf5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq='epoch',
    monitor='val_loss',
    mode='min',
    save_best_only=False)

# Fit the model
history = model.fit(dataset,epochs=EPOCHS,callbacks=[model_checkpoint])


Epoch 1/20
136851/136851 [==============================] - 10397s 76ms/step - loss: 1.3428
Epoch 2/20
136851/136851 [==============================] - 10362s 76ms/step - loss: 1.1128
Epoch 3/20
136851/136851 [==============================] - 10386s 76ms/step - loss: 1.0804
Epoch 4/20
136851/136851 [==============================] - 10365s 76ms/step - loss: 1.0646
Epoch 5/20
136851/136851 [==============================] - 10365s 76ms/step - loss: 1.0524
Epoch 6/20
136851/136851 [==============================] - 10364s 76ms/step - loss: 1.0425
Epoch 7/20
136851/136851 [==============================] - 10359s 76ms/step - loss: 1.0354
Epoch 8/20
136851/136851 [==============================] - 10370s 76ms/step - loss: 1.0297
Epoch 9/20
136851/136851 [==============================] - 10370s 76ms/step - loss: 1.0240
Epoch 10/20
136851/136851 [==============================] - 10381s 76ms/step - loss: nan
Epoch 11/20
136851/136851 [==============================] - 10363s 76ms/step - lo

##### Start Training Model from Saved Checkpoint
Training the model takes quite some time. A checkpoint is saved every epoch so the code below will allow you to resume training from a checkpoint

In [ ]:
#restart training from saved checkpoint
new_model = load_model('checkpoints/weights.20.hdf5')

# Save a checkpoint after every epoch
EPOCHS = 1
checkpoint_filepath = 'checkpoints/weights.{epoch:02d}.hdf5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq='epoch',
    monitor='val_loss',
    mode='min',
    save_best_only=False)

#Fit the model
history = new_model.fit(dataset,epochs=EPOCHS,callbacks=[model_checkpoint])


In [ ]:
# Save model
model.save('model.h5')

#### Load a saved model and generate a Tweet

In [2]:
# Create a datahelper object and designate the input file
dh = DataHelper(file_name='tweets_11-06-2020.csv')

# Prep the raw data to create the tweet file
dh.prep_raw_data(start_date='2020-08-01', end_date='2020-11-19')

# Print the number of tweets
print('The number of Tweets sent by Trump during the period is {}'.format(dh.num_tweets))

Data processing complete.
The number of Tweets sent by Trump during the period is 2362


In [19]:
# re-create the tokenizer
# Tokenize the text and create the dataset
dataset, tok = dh.create_tokenizer('inputdata/clean_tweet.txt')
print('The number of unique characters is {0:,} and the dataset size is {1:,} document(s).' \
      ' The number of windows in the dataset for processing is {2:,}.'.format(dh.num_unique_chars,dh.dataset_size,dh.num_data_windows))

# restore saved model for inferencing
mh = ModelHelper(epochs=1, tokenizer=tok)
new_model = mh.restore_model('weights.10.hdf5')

Dataset and tokenizer creation complete.
The number of unique characters is 75 and the dataset size is 1 document(s). The number of windows in the dataset for processing is 273,569.


##### Generate some tweets

In [20]:
# Create a long sequence of text
print(mh.create_tweet(text='harris', model=new_model, n_chars=140, temperature=0.2))

harris


In [21]:
print(len(mh.get_next_char(text='tr', model=new_model)))

0


In [22]:
x = mh.preprocess_pred('tr',tok)

In [23]:
print(x)

tf.Tensor(
[[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0.]]], shape=(2, 1, 75), dtype=float32)


In [24]:
y_proba = new_model.predict(x)[0, -1:, :]
print(y_proba)

[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan]]


In [16]:
print(x[0,-1:,:])

tf.Tensor(
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]], shape=(1, 75), dtype=float32)
